Use a CNN to classify trimmed spectra


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Read in individual datacubes and combine into a single dataframe

from astropy.table import Table
import pandas as pd

df = pd.DataFrame()

sightline_list = ['J010619+004823','J012403+004432','J013340+040059','J013724-422417','J015741-010629']#,
                  #'J020944+051713','J024401-013403']#,'J033413-161205','J033900-013318','J094932+033531',
                  #'J095852+120245']#,'J102009+104002','J111008+024458','J111113-080401','J120917+113830',
                  #'J123055-113909','J124957-015928','J133254+005250','J142438+225600','J162116-004251',
                  #'J193957-100241','J200324-325145','J205344-354652','J221527-161133','J230301-093930',
                  #'J231543+145606','J233446-090812','J234913-371259']


for sightline in sightline_list:
    table = Table.read('/home/emma/Documents/laeML/data/catalogues/individual_cubes/{}_cubex_data.fits'.format(sightline))
    table['sightline'] = sightline
    pandas_df = table.to_pandas()
    df = pd.concat([df,pandas_df])



In [3]:
# Cut to SNR > 7 as we haven't looked at all the sources below this
df = df[df.SNR >= 7]

In [4]:
# Check number of LAEs
type_str = df['type'].str.decode("utf-8")
nLAE = type_str.str.contains('LAE').sum()
print(nLAE)

172


In [5]:
# Create new column to show if entry is an LAE
df['isLAE'] = np.where(type_str.str.contains('LAE'), 1, 0)

We want to use 1D flux arrays from the trimmed spectra as input to our CNN (i.e. X) while is_LAE is the target (i.e. Y)

In [6]:
y=df.isLAE

Need to read in the flux columns from the trimmed arrays and combine into a dataframe to use as X

In [7]:
"""
#create empty list
xlist = []

from astropy.io import fits

#loop over entries, find the trimmed spectrum, add to dataframe
for row in df.itertuples(index=True, name='Pandas'):
    trimmed_file = '/home/emma/Documents/laeML/data/trimmed_spec/{}/spec_trim_id{}_-100A_+100A.fits'.format(row.sightline,row.id)
    
    #read in spectrum
    try:
        with fits.open(trimmed_file) as spec:
            flx = spec[0].data
            xlist.append(flx)
    except:
        print('Couldn\'t find: ' +str(row.sightline) + ' id' + str(row.id))
        continue
        
    
#convert list to dataframe. This is quicker than growing a dataframe row-wise
x = pd.DataFrame(xlist)
"""

"\n#create empty list\nxlist = []\n\nfrom astropy.io import fits\n\n#loop over entries, find the trimmed spectrum, add to dataframe\nfor row in df.itertuples(index=True, name='Pandas'):\n    trimmed_file = '/home/emma/Documents/laeML/data/trimmed_spec/{}/spec_trim_id{}_-100A_+100A.fits'.format(row.sightline,row.id)\n    \n    #read in spectrum\n    try:\n        with fits.open(trimmed_file) as spec:\n            flx = spec[0].data\n            xlist.append(flx)\n    except:\n        print('Couldn't find: ' +str(row.sightline) + ' id' + str(row.id))\n        continue\n        \n    \n#convert list to dataframe. This is quicker than growing a dataframe row-wise\nx = pd.DataFrame(xlist)\n"

In [9]:
#instead of creating x from each file individually, read it in from a prepared file
x=pd.read_csv("/home/emma/Documents/laeML/data/trimmed_spectra_dataframe.csv",index_col=0)


In [10]:
x.head()

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
0,590.384766,454.941467,354.345154,328.243042,5.348419,70.211662,235.562622,207.469299,137.152069,109.381187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,452.941223,427.720490,435.023926,538.362366,448.548431,518.894287,402.824951,319.419006,319.533020,267.410187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,237.862305,131.504059,196.990021,375.919373,264.392822,225.224884,189.368408,158.257416,97.674271,99.918762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,371.443054,337.741241,242.179245,256.283417,389.117188,477.863403,372.696228,448.540527,375.683411,559.601074,...,290.749451,296.189941,207.545380,299.615936,240.926071,307.433838,527.028931,481.803894,500.537750,474.636536
4,47.715321,-27.427366,42.573402,34.838043,89.349220,-45.284454,-17.226364,-45.073406,23.561562,-54.283142,...,114.334274,35.456001,-106.346481,25.491888,45.191193,45.270767,-9.925606,-58.586670,39.541794,29.653870


In [11]:
x=x.fillna(0)

In [12]:
x = np.array(x).reshape(x.shape[0], x.shape[1], 1)
print(x.shape)

(3846, 160, 1)


In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.15)

In [14]:
nLAE_trainSample = len(y_train[y_train==1])
print(nLAE_trainSample)

146


In [15]:
#create simple test data
#import random
#xfake = np.empty(x.shape)
#yfake=np.empty(y.shape)
#for i in range(len(xfake_test)):
#    k=random.randint(0, 1)
#    xfake[i] = k
#    yfake[i] = k
    

In [16]:
#xfake_train, xfake_test, yfake_train, yfake_test=train_test_split(xfake, yfake, test_size=0.15)

In [17]:
#Create simple CNN

#import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(160,1)))
model.add(Dense(16, activation="relu", input_shape=(160,1)))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))


In [18]:
model.compile(loss = 'binary_crossentropy', 
     optimizer = "adam",               
    metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 159, 64)           192       
                                                                 
 dense (Dense)               (None, 159, 16)           1040      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 79, 16)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1264)              0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1265      
                                                                 
Total params: 2,497
Trainable params: 2,497
Non-trainable params: 0
______________________________________________________

In [19]:
model.fit(x_train, y_train,
          batch_size=100,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/30
33/33 [==============================] - 1s 15ms/step - loss: 1.0975 - accuracy: 0.9324 - val_loss: 0.3476 - val_accuracy: 0.9515
Epoch 2/30
33/33 [==============================] - 0s 9ms/step - loss: 0.3838 - accuracy: 0.9367 - val_loss: 0.1849 - val_accuracy: 0.9463
Epoch 3/30
33/33 [==============================] - 0s 11ms/step - loss: 0.2144 - accuracy: 0.9428 - val_loss: 0.1518 - val_accuracy: 0.9463
Epoch 4/30
33/33 [==============================] - 0s 10ms/step - loss: 0.1477 - accuracy: 0.9495 - val_loss: 0.1400 - val_accuracy: 0.9376
Epoch 5/30
33/33 [==============================] - 0s 10ms/step - loss: 0.1257 - accuracy: 0.9538 - val_loss: 0.1474 - val_accuracy: 0.9376
Epoch 6/30
33/33 [==============================] - 0s 10ms/step - loss: 0.1146 - accuracy: 0.9523 - val_loss: 0.1296 - val_accuracy: 0.9480
Epoch 7/30
33/33 [==============================] - 0s 10ms/step - loss: 0.1085 - accuracy: 0.9544 - val_loss: 0.1293 - val_accuracy: 0.9549
Epoch 8/30
33/

In [20]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.185048446059227
Test accuracy: 0.9376083016395569


!!!!!!!!!!!!!!!! SHOULD DO TEST ON SEPARATE DATA NOT THE STUFF USED FOR VALIDATION !!!!!!

In [21]:
ypred = model.predict(x_test)
ypred = np.array(ypred)

for yi in range(len(ypred)):
    if ypred[yi]<0.5:
        ypred[yi] = 0
    else:
        ypred[yi] = 1

19/19 [==============================] - 0s 2ms/step


In [22]:
y_test=np.array(y_test)
nLAE_testsample = len(y_test[y_test==1])
print(nLAE_testsample)

26


In [23]:
isLAE_and_predLAE,isLAE_and_NotpredLAE,NotLAE_and_predLAE, NotLAE_and_NotpredLAE = 0,0,0,0

for i in range(len(y_test)):
    if (y_test[i] == 1) & (ypred[i][0]==1):
        isLAE_and_predLAE += 1 # Number of True LAEs predicted to be LAEs
    elif (y_test[i] == 1) & (ypred[i][0]!=1):
        isLAE_and_NotpredLAE += 1 # Number of True LAEs bot predicted to be LAEs
    if (y_test[i] != 1) & (ypred[i][0]==1):
        NotLAE_and_predLAE += 1 # Number of not LAEs predicted to be LAEs
    if (y_test[i] != 1) & (ypred[i][0] != 1):
        NotLAE_and_NotpredLAE += 1 # Number of not LAEs predicted to not be LAEs

In [24]:
print(isLAE_and_predLAE,isLAE_and_NotpredLAE,NotLAE_and_predLAE,NotLAE_and_NotpredLAE)

10 16 20 531


In [ ]:
ypred